<a href="https://colab.research.google.com/github/farfaness/Scraping/blob/master/linternaute_web_scraping_selenium_beautifulsoup4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encyclopédie des villes de France

## Getting started

In [0]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [0]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from unidecode import unidecode
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
import time

In [0]:
!pip install Unidecode
!pip install beautifulsoup4

## Liste des hrefs des villes de France (source: L'encyclopédie des villes sur l'internaute) with BeautifulSoup

In [0]:
#liste des pages de l'internaute contenant les hrefs des villes de France
liste_pages = [("http://www.linternaute.com/ville/index/villes?page=" + "{0:03}".format(x)) for x in list(range(1, 710))]

#Loop sur ces pages pour obtenir les hrefs
villes_FR = pd.DataFrame(columns = ["href"])

for page in liste_pages : 
    
    html_page = urlopen(page)
    soup = BeautifulSoup(html_page)
    liste_href_villes = []
    
    [liste_href_villes.append("http://www.linternaute.com" + (li.find('a')['href'])) for ul in soup.find_all('ul', class_='bloc size1of2 odSquareList') for li in ul.find_all('li')]

    # li.find('a').get_text() pour avoir le texte dans la balise (après le href) au lieu de href
                
    villes_FR = pd.concat([villes_FR, pd.DataFrame(liste_href_villes, columns = ["href"])], ignore_index=True, axis = 0)

In [0]:
from google.colab import drive
import shutil
drive.mount("/content/drive", force_remount=True)

In [0]:
#villes_FR.to_csv('C:\\Users\\utilisateur\\Documents\\web_scrapping\\villes_FR.csv')

# Save the Dataframe to csv on google drive
villes_FR.to_csv("/content/villes_FR.csv")

# Déplacer le fichier de content vers le dossier datasets dans My Drive
shutil.move("/content/villes_FR.csv", "/content/drive/My Drive/Datasets/villes_FR.csv") 

'/content/drive/My Drive/Datasets/villes_FR.csv'

## Scrap infos on French cities

In [0]:
#Create 355 dataframes of cities list
i = 0
nb_cities_per_threading = 100
start = 0
end = nb_cities_per_threading

while i < (len(villes_FR)//nb_cities_per_threading) : 
    globals()[('villes_FR' + str(i))] = villes_FR.iloc[start:end, :]
    start += nb_cities_per_threading
    end += nb_cities_per_threading
    i +=1
    
#For the last cut 
end = start + (len(villes_FR)%nb_cities_per_threading)
globals()[('villes_FR' + str(i))] = villes_FR.iloc[start:end, :]

In [0]:
#Create class to use multithreading to scrap population for each french city
from threading import Thread, RLock
verrou = RLock()

class Scrapeur(Thread):
    """Thread chargé de scraper la population de chaque villes"""

    def __init__(self, villes_FR):
        Thread.__init__(self)
        self.villes_FR = villes_FR
        
    def run(self):
        """Code à exécuter pendant l'exécution du thread."""
        globals()[('pop_villes' + str(i))] = pd.DataFrame(columns = ["href", "pop"])
        browser = webdriver.Chrome('chromedriver', options=options)
        
        for globals()[('city' + str(i))] in self.villes_FR['href']:
            browser.get(globals()[('city' + str(i))])

            #Get demographic informations
            # Demographic infos on the front page
            liste_pop = browser.find_elements_by_class_name('odTable.odTableAuto')
            time.sleep(1)
            liste_pop_str = [liste_pop.text for liste_pop in liste_pop]
            globals()[('nb_hab' + str(i))] = re.findall('.+\) (\d*\s*\d*\s*\d*) hab\. .+', liste_pop_str[1]) #find nb pop
            
            #Store nb pop in dataframe pop_villes_i
            globals()[('pop_villes' + str(i))] = globals()[('pop_villes' + str(i))].append({'href' : globals()[('city' + str(i))], 'pop': globals()[('nb_hab' + str(i))]}, ignore_index=True)
            
        browser.quit()
        
        return globals()[('pop_villes' + str(i))]

In [0]:
# To initiate, run only once
all_pop_villes = pd.DataFrame(columns = ["href", "pop"])

iterations = 0 #start for iterations
depart = 0
nb_threads = 10 #nbr threads per iteration
fin = nb_threads

In [0]:
#if you already registred data in the csv all_pop_villes use that instead
all_pop_villes = pd.read_csv("/content/drive/My Drive/Datasets/villes_pop_FR.csv")

nbr_iterations_already_run = 2 #nbr of iterations already run successfuly

iterations = nbr_iterations_already_run + 1 #new_start for iterations
nb_threads = 10 #nbr threads per iteration
depart = nb_threads * nbr_iterations_already_run
fin = depart + nb_threads

In [0]:
# Run multithreadings

end_iterations_to_run = 2 # need to go to 23; Pour un total de 355 datasets

while iterations < end_iterations_to_run : # (355//nb_threads): # soit 23; Pour un total de 355 datasets, fait plus bas une 24ème iteration avec 5 threads 
    nb_villes_FR_datasets = list(range(depart, fin))
    
    if iterations < (355//nb_threads) : 
        #Lancer x threads à la fois
        for i in nb_villes_FR_datasets:
            #Creation du thread
            time.sleep(1)
            globals()[('thread_' + str(i))] = Scrapeur(globals()[('villes_FR' + str(i))])   
            time.sleep(2)
            # Lancement du thread
            globals()[('thread_' + str(i))].start()
            print('Thread {} iniated !! hihi' .format(i))
            
            
        for i in nb_villes_FR_datasets:
            #Cloture des x threads
            globals()[('thread_' + str(i))].join()
        
        for i in nb_villes_FR_datasets:
            #Update the dataframe with the threads scrap infos
            time.sleep(1)
            all_pop_villes = pd.concat([all_pop_villes, globals()[('pop_villes' + str(i))]], axis = 0)
            del globals()[('pop_villes' + str(i))]
            print('Thread {} completed !! :D' .format(i))
    
    elif iterations == (355//nb_threads) :
        nb_threads = 355 % nb_threads
        fin = depart + nb_threads
        #Lancer x threads à la fois
        for i in nb_villes_FR_datasets:
            #Creation du thread
            time.sleep(1)
            globals()[('thread_' + str(i))] = Scrapeur(globals()[('villes_FR' + str(i))])  
            time.sleep(2)
            # Lancement du thread
            globals()[('thread_' + str(i))].start()
            print('Thread {} iniated !! hihi' .format(i))
            
        
        for i in nb_villes_FR_datasets:
            #Cloture des x threads
            globals()[('thread_' + str(i))].join()
        
        for i in nb_villes_FR_datasets:
            #Update the dataframe with the threads scrap infos
            time.sleep(1)
            all_pop_villes = pd.concat([all_pop_villes, globals()[('pop_villes' + str(i))]], axis = 0)
            del globals()[('pop_villes' + str(i))]
            print('Thread {} completed !! :D' .format(i))
            
    depart = fin
    fin += nb_threads 
    iterations +=1


In [0]:
# Sanity check
print(len(all_pop_villes)) # 20 threads de 100
all_pop_villes.head()

1839


,href,pop
0,http://www.linternaute.com/ville/archigny/vill...,[173]
1,http://www.linternaute.com/ville/abainville/vi...,[213]
2,http://www.linternaute.com/ville/agassac/ville...,[3 021]
3,http://www.linternaute.com/ville/aimargues/vil...,[114]
4,http://www.linternaute.com/ville/agde/ville-34003,[5 413]


In [0]:
# Mount to save to google drive
drive.mount("/content/drive", force_remount=True)

In [0]:
#Save the Dataframe to csv on google drive
all_pop_villes.to_csv("/content/villes_pop_FR.csv")

# Déplacer le fichier de content vers le dossier datasets dans My Drive
shutil.move("/content/villes_pop_FR.csv", "/content/drive/My Drive/Datasets/villes_pop_FR.csv") 

'/content/drive/My Drive/Datasets/villes_pop_FR.csv'

# Filter : Keep French cities with more than 50 000 hab.

In [0]:
#Read the dataframe in memory
data = pd.read_csv("/content/drive/My Drive/Datasets/villes_pop_FR.csv")

In [0]:
print(len(data))
data.drop(columns = 'Unnamed: 0', inplace = True)
data.head()

1839


,href,pop
0,http://www.linternaute.com/ville/archigny/vill...,['173']
1,http://www.linternaute.com/ville/abainville/vi...,['213']
2,http://www.linternaute.com/ville/agassac/ville...,['3 021']
3,http://www.linternaute.com/ville/aimargues/vil...,['114']
4,http://www.linternaute.com/ville/agde/ville-34003,['5 413']


In [0]:
# Delete the lignes where pop wasn't write down
for index in data.index.values :
  if data["pop"][index] == '[]' : data.drop([index], inplace=True)
    
print(len(data))

1799


In [0]:
# Use ville_FR.href to get the list of cities not yet scraped
ville_FR = pd.read_csv("/content/drive/My Drive/Datasets/villes_FR.csv")
#not_scraped = villes_FR.loc[villes_FR["href"] in new_data['href']]

In [0]:
# Transform pop into int
new_data = data.copy()
for index in data.index.values:
  new_data['pop'][index] = (int(''.join(c for c in data['pop'][index] if c.isdigit())))

In [0]:
# Filter cities with more than 50 000 habitants
big_cities = new_data.loc[new_data['pop']>= 50000 ]

In [0]:
print(len(big_cities))

15


In [0]:
big_cities.head(15)

,href,pop
45,http://www.linternaute.com/ville/arcins/ville-...,125694
199,http://www.linternaute.com/ville/agny/ville-62013,132874
333,http://www.linternaute.com/ville/anould/ville-...,151520
336,http://www.linternaute.com/ville/ahun/ville-23001,142668
480,http://www.linternaute.com/ville/arganchy/vill...,68462
571,http://www.linternaute.com/ville/alba-la-romai...,74875
655,http://www.linternaute.com/ville/angoume/ville...,110388
704,http://www.linternaute.com/ville/argenton-sur-...,61711
840,http://www.linternaute.com/ville/aix-en-proven...,142668
856,http://www.linternaute.com/ville/ajaccio/ville...,68462
